In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse, root_mean_squared_error as rmse

In [ ]:
jan_2023 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'
feb_2023 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet'

df_jan = pd.read_parquet(jan_2023)
df_feb = pd.read_parquet(feb_2023)

print(f'Dataset dimensions for January -> {df_jan.shape=}')
print(f'Dataset dimensions for February -> {df_feb.shape=}')

In [ ]:
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)

print(f'Standard deviation for January -> {(df_jan["duration"].std()):.2f}')

Standard deviation for January -> 42.59


In [ ]:
before = df_jan.shape[0]

df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]
after = df_jan.shape[0]

print(f'New fraction records -> {(after / before)*100:.0f}%')

New fraction records -> 98%


In [ ]:
categorical = ['PULocationID', 'DOLocationID']
df_jan[categorical] = df_jan[categorical].astype(str)

records = df_jan[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(records)

print(f'Dimensionality -> {X_train.shape=}')

In [ ]:
y_train = df_jan['duration'].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
mse_calculated = mse(y_train, y_pred)
rmse_calculated = rmse(y_train, y_pred)

print(f'(MSE) Error calculated with Linear Regression --> {mse_calculated:.4f}')
print(f'(RMSE) Error calculated with Linear Regression --> {rmse_calculated:.4f}')

(MSE) Error calculated with Linear Regression --> 58.5112
(RMSE) Error calculated with Linear Regression --> 7.6493


**Final Question ->**

In [2]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_parquet(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def data_pipeline(dataset, dv, type='train'):
    categorical = ['PULocationID', 'DOLocationID']

    records = dataset[categorical].to_dict(orient='records')

    if type == 'train':
        X = dv.fit_transform(records)
    else:
        X = dv.transform(records)

    return X

In [3]:
jan_2023 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'
feb_2023 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet'

df_train = read_dataframe(jan_2023)
print(f'Train Dataset dimensions -> {df_train.shape=}')

df_val = read_dataframe(feb_2023)
print(f'Validation Dataset dimensions -> {df_val.shape=}')

Train Dataset dimensions -> df_train.shape=(3009173, 20)
Validation Dataset dimensions -> df_val.shape=(2855951, 20)


In [4]:
dv = DictVectorizer()

X_train = data_pipeline(df_train, dv, type='train')
X_val = data_pipeline(df_val, dv, type='val')

y_train = df_train['duration']
y_val = df_val['duration']

In [5]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
mse_calculated = mse(y_val, y_pred)
rmse_calculated = rmse(y_val, y_pred)

print(f'(MSE) Error calculated with Linear Regression --> {mse_calculated:.4f}')
print(f'(RMSE) Error calculated with Linear Regression --> {rmse_calculated:.4f}')

(MSE) Error calculated with Linear Regression --> 177.4941
(RMSE) Error calculated with Linear Regression --> 13.3227
